<a href="https://colab.research.google.com/github/VesalAhsani/Deep-learning-with-PyTorch/blob/main/03_PyTorch_Notebook_To_Scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructor: Dr. Vesal Ahsani

This section answers the question

**"how do I turn my notebook code (Jupyter Notebook or Google Colab Notebook) into Python scripts?"**

## What is script mode?

**Script mode** uses [Jupyter Notebook cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) (special commands) to turn specific cells into Python scripts.

For example if you run the following code in a cell, you'll create a Python file called `hello_world.py`:

```
%%writefile hello_world.py
print("hello world, machine learning is fun!")
```

You could then run this Python file on the command line with:

```
python hello_world.py

>>> hello world, machine learning is fun!
```

The main cell magic we're interested in using is `%%writefile`.

Putting `%%writefile filename` at the top of a cell in Jupyter or Google Colab will write the contents of that cell to a specified `filename`.




As an example, if you find a PyTorch project on GitHub, it may be structured in the following way:

```
pytorch_project/
├── pytorch_project/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── model_1.pth
│   └── model_2.pth
└── data/
    ├── data_folder_1/
    └── data_folder_2/
```

Here, the top level directory is called `pytorch_project` but you could call it whatever you want.

Inside there's another directory called `pytorch_project` which contains several `.py` files, the purposes of these may be:
* `data_setup.py` - a file to prepare data (and download data if needed).
* `engine.py` - a file containing various training functions.
* `model_builder.py` or `model.py` - a file to create a PyTorch model.
* `train.py` - a file to leverage all other files and train a target PyTorch model.
* `utils.py` - a file dedicated to helpful utility functions.

And the `models` and `data` directories could hold PyTorch models and data files respectively (though due to the size of models and data files, it's unlikely you'll find the *full* versions of these on GitHub, these directories are present above mainly for demonstration purposes).

> **Note:** There are many different ways to structure a Python project and subsequently a PyTorch project. This isn't a guide on *how* to structure your projects, only an example of how you *might* come across PyTorch projects.

## 1. Get the data

In [4]:
import requests
from pathlib import Path

In [6]:
data_path = Path("data/")
data_path.mkdir(parents=True, exist_ok=True)

with open("data/food101_pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/VesalAhsani/Deep-learning-with-PyTorch/raw/main/data/food101_pizza_steak_sushi.zip")
  f.write(request.content)

In [7]:
import zipfile

with zipfile.ZipFile("data/food101_pizza_steak_sushi.zip", "r") as zz:
  zz.extractall("data/food101_pizza_steak_sushi")

## 2. Create datasets and dataloaders

data -> ImageFolder -> DataLoader

In [12]:
new_folder = Path("food101_project").mkdir(parents=True, exist_ok=True)

In [44]:
%%writefile food101_project/data_setup.py

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_transform = transforms.Compose([transforms.Resize(size=(64,64)),
                                    transforms.ToTensor()])

train_dir = "data/food101_pizza_steak_sushi/train"
test_dir = "data/food101_pizza_steak_sushi/test"

train_dataset = datasets.ImageFolder(root=train_dir,
                                     transform=data_transform)
test_dataset = datasets.ImageFolder(root=test_dir,
                                    transform=data_transform)

class_names = train_dataset.classes

BATCH_SIZE=32
NUM_WORKERS = 1

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)
test_dataloader = DataLoader(dataset=test_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=NUM_WORKERS)

Overwriting food101_project/data_setup.py


## 3. Making a model

In [90]:
%%writefile food101_project/model_builder.py

import torch
from torch import nn

class Food101Model(nn.Module):
  def __init__(self, input_shape, hidden_units, output_shape):
    super().__init__()

    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features= 1690,
                  out_features=output_shape)
    )
  
  def forward(self, x):
    return self.classifier(self.block_2(self.block_1(x)))

Overwriting food101_project/model_builder.py


## 4. Create functions for training and testing loops

In [71]:
%%writefile food101_project/engine.py
import torch
from torch import nn
from tqdm.auto import tqdm
def train_step(model, dataloader, loss_fn, accuracy_fn, optimizer, device):
  
  model.train()
  train_loss, train_acc = 0, 0

  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy_fn(y_pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)
  return train_loss, train_acc

def test_step(model, dataloader, loss_fn, accuracy_fn, device):
  
  model.eval()
  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch , (X,y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      test_pred = model(X)
      loss = loss_fn(test_pred, y)
      test_loss += loss
      test_acc += accuracy_fn(test_pred, y)
  
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)
    return test_loss, test_acc



results = {"train_loss": [],
           "train_acc": [],
           "test_loss": [],
           "test_acc": []
}

def train(model, train_dataloader, test_dataloader, loss_fn, accuracy_fn, optimizer, epochs, device):

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       accuracy_fn = accuracy_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                       dataloader=test_dataloader,
                                       loss_fn=loss_fn,
                                       accuracy_fn = accuracy_fn,
                                       device=device)
    
    print(f"Epochs: {epoch+1} | train_loss: {train_loss} | train_acc: {train_acc} | test_loss: {test_loss} | test_acc: {test_acc}")

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_loss)
    results["test_loss"].append(train_loss)
    results["test_acc"].append(train_loss)

  return results

Overwriting food101_project/engine.py


## 5. Save a model

In [81]:
%%writefile food101_project/utils.py

import torch
from pathlib import Path

def save(model, save_path):
  """
  save_path should include .pth or .pt at the end.
  """  
  torch.save(obj=model.state_dict(),
             f=save_path)

Overwriting food101_project/utils.py


## 6. Train, evaluate and save the model, All together

Let's combine all of our modular files into a single script `train.py`.

This will allow us to run all of the functions we've written with a single line of code on the command line:

`python food101_pizza_steak_sushi/train.py`

Or if we're running it in a notebook:

`!python food101_pizza_steak_sushi/train.py`

We'll go through the following steps:
1. Import the various dependencies, namely `torch`, `os`, `torchvision.transforms` and all of the scripts from the `food101_pizza_steak_sushi` directory, `data_setup`, `engine`, `model_builder`, `utils`.
  * **Note:** Since `train.py` will be *inside* the `food101_pizza_steak_sushi` directory, we can import the other modules via `import ...` rather than `from food101_pizza_steak_sushi import ...`.
2. Setup various hyperparameters such as batch size, number of epochs, learning rate and number of hidden units (these could be set in the future via [Python's `argparse`](https://docs.python.org/3/library/argparse.html)).
3. Setup the training and test directories.
4. Setup device-agnostic code.
5. Create the necessary data transforms.
6. Create the DataLoaders using `data_setup.py`.
7. Create the model using `model_builder.py`.
8. Setup the loss function and optimizer.
9. Train the model using `engine.py`.
10. Save the model using `utils.py`. 

In [93]:
%%writefile food101_project/train.py

import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics import Accuracy

import data_setup, engine, model_builder, utils

# Setup hyperparameters
BATCH_SIZE=32
NUM_WORKERS = 1
NUM_EPOCHS = 20
HIDDEN_UNITS=10
LEARNING_RATE=0.001

# Setup directories
train_dir = "data/food101_pizza_steak_sushi/train"
test_dir = "data/food101_pizza_steak_sushi/test"

# setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Pipeline


model_0 = model_builder.Food101Model(input_shape=3, 
                                     hidden_units=HIDDEN_UNITS, 
                                     output_shape=len(data_setup.class_names)
                                     ).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                             lr=LEARNING_RATE)
accuracy_fn = Accuracy().to(device)

engine.train(model=model_0,
             train_dataloader=data_setup.train_dataloader,
             test_dataloader=data_setup.test_dataloader,
             loss_fn=loss_fn,
             accuracy_fn=accuracy_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device
             )

utils.save(model=model_0,
           save_path = "model/01_scrip_version_food101_sample.pth")

Overwriting food101_project/train.py


In [94]:
!python food101_project/train.py

  0% 0/20 [00:00<?, ?it/s]Epochs: 1 | train_loss: 1.1005330085754395 | train_acc: 0.27734375 | test_loss: 1.1083883047103882 | test_acc: 0.2708333432674408
  5% 1/20 [00:02<00:42,  2.24s/it]Epochs: 2 | train_loss: 1.1059037446975708 | train_acc: 0.3125 | test_loss: 1.120385766029358 | test_acc: 0.1979166716337204
 10% 2/20 [00:04<00:40,  2.22s/it]Epochs: 3 | train_loss: 1.0856938362121582 | train_acc: 0.42578125 | test_loss: 1.0861849784851074 | test_acc: 0.4422348439693451
 15% 3/20 [00:06<00:37,  2.21s/it]Epochs: 4 | train_loss: 1.068902611732483 | train_acc: 0.44140625 | test_loss: 1.046971321105957 | test_acc: 0.5227272510528564
 20% 4/20 [00:08<00:35,  2.20s/it]Epochs: 5 | train_loss: 1.0447355508804321 | train_acc: 0.390625 | test_loss: 0.9936234354972839 | test_acc: 0.5539772510528564
 25% 5/20 [00:11<00:33,  2.21s/it]Epochs: 6 | train_loss: 0.9125605821609497 | train_acc: 0.59375 | test_loss: 1.0024447441101074 | test_acc: 0.38257575035095215
 30% 6/20 [00:13<00:31,  2.22s/it]E

We see that we could train our model by `python train.py` in the command line or `!python train.py` in cell.

And if we wanted to, we could adjust our `train.py` file to use argument flag inputs with Python's argparse module, this would allow us to provide different hyperparameter settings like previously discussed:

`python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS`



**Note:**

we should import required libararies in each of the `.py` files when we write our code. In other words, importing all packages in the `train.py` file is not enough!

BTW, our model results are not satisfactory. We should work on it later to improve!